<a href="https://colab.research.google.com/github/alvarorl231/ResearchMethods/blob/main/Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scrapping

## Imports, etc.

In [ ]:
!pip install BeautifulSoup4
!pip install requests

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Declaración de Variables

In [3]:
# Lista que contendrá toda la información
frases = []

# Lista que almacenara el contenido sensible
sensitive=[]
sensitive_filtered=[]

# Lista que almacenara el contenido NO sensible
nonsensitive=[]

# Keywords
keywords = ['ametralladora', 'arma', 'ataque', 'batallón', 'bomba', 
             'capitán', 'comandante', 'compañía', 'dispar', 'herido',
             'muerto', 'munición', 'municion', 'oficial', 'tiro']

filtered_keywords = ['ametralladora', 'ametralladoras',
                     'arma', 'arma', 'armas', 'armado', 'armados', 'armamento'
                     'ataque', 'ataques', 
                     'batallón', 
                     'bomba', 'bombardeos', 'bombardeo', 'bombardean', 'bombas', 'bombardea', 'bombardeará', 'bombardeaba', 
                     'capitán', 
                     'comandante', 'comandantes',
                     'compañía', 'compañías',
                     'disparo', 'disparos', 'dispara','dispararon',
                     'herido', 'heridos',
                     'muerto', 'muertos',
                     'munición', 'municiones',
                     'oficial', 
                     'tiro', 'tiroteo', 'tiro', 'tiroteo', 'tiroteos', 'tiroteado', 'tiroteos', 'tiros', 'tiroteados']             


## Obteción de la Información

Los periódicos elegidos la propiedad de que muestran las noticias en webs con urls del estilo:
```
https://elpais.com/noticias/terrorismo-internacional/
https://elpais.com/noticias/terrorismo-internacional/1/
https://elpais.com/noticias/terrorismo-internacional/2/
.
.
.
.
https://elpais.com/noticias/terrorismo-internacional/n/
```
Esto facilita mucho el scrapping, ya que al conseguir extraer los datos de una página, para las n siguientes solo debes aplicar un bucle.

De ese modo, la función `get_tags` nos ayudara a agilizar el proceso y a obtener una mayor limpieza en el código

In [4]:
def get_tags (url, id):

 result = requests.get(url)

 doc = BeautifulSoup(result.text, "html.parser")

 if (id == 'span'):
   html_tags = doc.find_all("span")
   print(f'span,{url}')
 elif (id == 'h1'):
   html_tags = doc.find_all("h1") 
   print(f'h1,{url}')
 else:
   html_tags = doc.find_all("h2")
   print(f'h2,{url}')

 return html_tags

### Periódico: [RRP (Perú)](https://rpp.pe/noticias/terrorismo)

In [ ]:
id = 'h2'

for i in range(1,6):
  if i == 1:
   rrp_url = "https://rpp.pe/noticias/terrorismo"
   tags = get_tags(rrp_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   rrp_new_url = f"https://rpp.pe/noticias/terrorismo?page={i}/"
   tags = get_tags(rrp_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

### Periódico: [La Voz (Argentina)](https://www.lavoz.com.ar/temas/terrorismo/)

In [ ]:
id = 'h2'

for i in range(1,46):
  if i == 1:
   lavoz_url = "https://www.lavoz.com.ar/temas/terrorismo/"
   tags = get_tags(lavoz_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   lavoz_new_url = f"https://www.lavoz.com.ar/temas/terrorismo/{i}/"
   tags = get_tags(lavoz_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

### Periódico: [Gestión (Perú)](https://gestion.pe/noticias/terrorismo/)

In [ ]:
id = 'h2'

for i in range(1,6):
  if i == 1:
   gestion_url = "https://gestion.pe/noticias/terrorismo/"
   tags = get_tags(gestion_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   gestion_new_url = f"https://gestion.pe/noticias/terrorismo/{i}/"
   tags = get_tags(gestion_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

### Periódico: [El Comercio (Perú)](https://elcomercio.pe/noticias/terrorismo/)

In [ ]:
id = 'h2'

for i in range(1,33):
  if i == 1:
   elcomerico_url = "https://elcomercio.pe/noticias/terrorismo/"
   tags = get_tags(elcomerico_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   elcomerico_new_url = f"https://elcomercio.pe/noticias/terrorismo/{i}/"
   tags = get_tags(elcomerico_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

### Periódico: [Heraldo](https://www.heraldo.es/tags/temas/terrorismo.html/)

In [ ]:
id = 'h1'

for i in range(1,11):
  if i == 1:
   heraldo_url = "https://www.heraldo.es/tags/temas/terrorismo.html/"
   tags = get_tags(heraldo_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   heraldo_new_url = f"https://www.heraldo.es/tags/temas/terrorismo.html/{i}/"
   tags = get_tags(heraldo_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

### Periódico: [Europa Press](https://www.europapress.es/temas/terrorismo)

In [ ]:
id = 'h2'

for i in range(1,301):
  if i == 1:
   elmundo_url = "https://www.europapress.es/temas/terrorismo/"
   tags = get_tags(elmundo_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   elmundo_new_url = f"https://www.europapress.es/temas/terrorismo/{i}"
   tags = get_tags(elmundo_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

### Periódico: [El Mundo](https://www.elmundo.es/t/te/terrorismo.html)

In [ ]:
id = 'h2'

for i in range(1,16):
  if i == 1:
   elmundo_url = "https://www.elmundo.es/t/te/terrorismo/"
   tags = get_tags(elmundo_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   elmundo_new_url = f"https://www.elmundo.es/t/te/terrorismo/pag{i}"
   tags = get_tags(elmundo_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)


### Periódico: [El País](https://elpais.com/ultimas-noticias/)

In [ ]:
id = 'h2'

for i in range(1,370):
  if i == 1:
   elpais_url = "https://elpais.com/noticias/terrorismo-internacional/"
   tags = get_tags(elpais_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   elpais_new_url = f"https://elpais.com/noticias/terrorismo-internacional/{i}/"
   tags = get_tags(elpais_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

In [ ]:
id = 'h2'

for i in range(1,370):
  if i == 1:
   elpais_url = "https://elpais.com/noticias/terrorismo/"
   tags = get_tags(elpais_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   elpais_new_url = f"https://elpais.com/noticias/terrorismo/{i}/"
   tags = get_tags(elpais_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

### Periódico: [20 Minutos](https://www.20minutos.es/minuteca/terrorismo/)

In [ ]:
id = 'h1'

for i in range(1,370):
  if i == 1:
   minutos_url = "https://www.20minutos.es/minuteca/terrorismo/"
   tags = get_tags(minutos_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   minutos_new_url = f"https://www.20minutos.es/minuteca/terrorismo/{i}/"
   tags = get_tags(minutos_new_url, id)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

## Selección de palabras sensibles

### Keyword `in` frase

Tras esto limpiamos la lista de frases obtenida anteriormente y la empleamos para buscar el contenido sensible

In [15]:
# Ahora vamos eliminar duplicados con set()
frases = list(set(frases))

# Por cada keyword busca en todos los elementos de la lista frases, si se da esta condición, se almacena.
for i in keywords:
  for j in range(len(frases)):
    if (i in frases[j]) or (i.capitalize() in frases[j]):
      sensitive.append(frases[j])

sensitive = list(set(sensitive))

### Keyword `==` palabra de cada frase

In [21]:
sensitive_filtered.clear()

In [22]:
# Por cada filtered_keywords busca en todos los elementos de la lista frases, si se da esta condición, se almacena.
for i in filtered_keywords:
  for j in range(len(frases)):
    aux_list = frases[j].split()
    for k in range(len(aux_list)):
     if (i == aux_list[k]) or (i.capitalize() == aux_list[k]):
      sensitive_filtered.append(frases[j])


sensitive_filtered = list(set(sensitive_filtered))

### Creación de la lista de frases no sensibles

In [17]:
nonsensitive = [i for i in frases if i not in sensitive]
nonsensitive = list(set(nonsensitive))

## Almacenar los datos en archivos `.csv`

In [23]:
# Todos los títulos de las noticias
df = pd.DataFrame(frases)
df.to_csv("all.csv", index=False)

# Todos los títulos SENSIBLES
df = pd.DataFrame(sensitive)
df.to_csv("sensitive.csv", index=False)

# Todos los títulos SENSIBLES FILTRADOS
df = pd.DataFrame(sensitive_filtered)
df.to_csv("sensitive_filtered.csv", index=False)

# Todos los títulos NO SENSIBLES

df = pd.DataFrame(nonsensitive)
df.to_csv("nonsensitive.csv", index=False)